#### Regression Analysis on Interest Rate Volatility

This notebook performs a regression analysis to model how central bank communication impacts the volatility of the term structure of futures interest rates.
 * It uses several linear regression specifications to examine the relationship between different aspects of the communication—such as the event of the release, the level of the sentiment indices (optimism and uncertainty), and dummies for pessimistic communication—and the volatility of interest rates across various maturities.

In [1]:
import pandas as pd
import numpy as np
import statsmodels.api as sm

from sklearn.linear_model import LassoCV
from sklearn.preprocessing import StandardScaler

In [2]:
INITIAL_DATE = "2003-06-26"

In [3]:
minutes_info = pd.read_excel("./data/raw/minutes_info.xlsx")
minutes_info = minutes_info[minutes_info["DataReferencia"] >= INITIAL_DATE]
minutes_info = minutes_info.sort_values("DataReferencia", ascending=True)

dates_ref = minutes_info["DataReferencia"]
detes_pub = minutes_info["DataPublicacao"]

In [4]:
futures_ir = pd.read_excel("./data/future_interest_rates/future_interest_rates.xlsx", index_col="Date")
futures_ir.index = pd.to_datetime(futures_ir.index)
futures_ir = futures_ir[futures_ir.index >= INITIAL_DATE]

futures_ir.sort_index(inplace=True)

futures_ir_diff = futures_ir.diff().dropna()

vertix_names = futures_ir_diff.columns.to_list()

#### Futures MA Volatility (5 bd)

In [5]:
def calc_vol(returns: pd.Series) -> float:
    vol = returns.std()
    return vol

mw_volatility_5 = futures_ir_diff.rolling(5).apply(calc_vol)
mw_volatility_5 = mw_volatility_5.dropna()

mw_volatility_5 = np.log(mw_volatility_5)

In [6]:
mw_volatility_5["dummy_minute_pub_date"] = 0
mw_volatility_5.loc[mw_volatility_5.index.isin(detes_pub), "dummy_minute_pub_date"] = 1

#### Regressions on optimism index

In [7]:
df_optimism = pd.read_excel("./data/processed/index_optimism.xlsx", index_col=0)
df_optimism = df_optimism.drop(columns=['minute'])
df_optimism.columns = [f"optimism_{i}" for i in range(len(df_optimism.columns))]

df_optimism_dif = df_optimism.diff()

In [8]:
df_reg_opt = mw_volatility_5.copy()
df_reg_opt = df_reg_opt.join(df_optimism_dif)
cols_optimism = [col for col in df_reg_opt.columns if 'optimism_' in col]
df_reg_opt[cols_optimism] = df_reg_opt[cols_optimism].fillna(0)

In [9]:
len(df_reg_opt)

5524

In [10]:
results_reg_opt = []
possible_y = vertix_names
all_x_reg_opt = cols_optimism + ["dummy_minute_pub_date"]
X_reg_opt_with_dummy = df_reg_opt[all_x_reg_opt]
X_reg_opt = X_reg_opt_with_dummy[cols_optimism]

scaler_reg_opt = StandardScaler()
X_reg_opt_scaled = scaler_reg_opt.fit_transform(X_reg_opt)

X_reg_opt_scaled = pd.DataFrame(X_reg_opt_scaled, index=X_reg_opt.index, columns=X_reg_opt.columns)

for maturity in possible_y:
    Y = df_reg_opt[maturity]

    lasso_cv_model = LassoCV(cv=10, random_state=100, max_iter=10000)
    lasso_cv_model.fit(X_reg_opt_scaled, Y)

    selected_coeffs_mask = lasso_cv_model.coef_ != 0
    selected_variables_names = X_reg_opt.columns[selected_coeffs_mask].tolist()

    X_selected = X_reg_opt_with_dummy[selected_variables_names + ["dummy_minute_pub_date"]]
    X_selected_sm = sm.add_constant(X_selected)

    ols_final_model = sm.OLS(Y, X_selected_sm).fit()

    all_var_names_with_const = X_selected_sm.columns.tolist()
    ols_params = pd.Series(ols_final_model.params, index=all_var_names_with_const)
    ols_std = pd.Series(ols_final_model.bse, index=all_var_names_with_const)
    ols_pvalues = pd.Series(ols_final_model.pvalues, index=all_var_names_with_const)

    results_reg_opt.append({
        'Vertix': maturity,
        'Selected_Alpha': lasso_cv_model.alpha_,
        'Coefs_OLS_Final': ols_params,
        'Std_OLS_Final': ols_std,
        'Pvalues_OLS_Final': ols_pvalues,
        'Adj_R_Sqrd_OLS_Final': ols_final_model.rsquared_adj
    })


In [11]:
for result in results_reg_opt:
    print(f"#### Results for Maturity (Vertex): {result['Vertix']}")
    print(f"Best alpha (λ) found by cross-validation: {result['Selected_Alpha']:.6f}")
    print(f"Adjusted R squared of the final OLS model: {result['Adj_R_Sqrd_OLS_Final']:.4f}")
            
    # --- Final OLS Model Results ---
    if not result['Coefs_OLS_Final'].empty:
        print("\n  Final OLS Results:")
        for var in result['Coefs_OLS_Final'].index:
            coef = result['Coefs_OLS_Final'][var]
            std_err = result['Std_OLS_Final'][var]
            pval = result['Pvalues_OLS_Final'][var]
            print(f"    - {var}: Coef = {coef:.6f}, Std. Err = {std_err:.6f}, P-value = {pval:.4f}")
    else:
        print("No variables selected.")
    print("-" * 60)

#### Results for Maturity (Vertex): v_21
Best alpha (λ) found by cross-validation: 0.013069
Adjusted R squared of the final OLS model: 0.0026

  Final OLS Results:
    - const: Coef = -4.562400, Std. Err = 0.012933, P-value = 0.0000
    - optimism_3: Coef = -14.487940, Std. Err = 8.029396, P-value = 0.0712
    - optimism_5: Coef = -17.132150, Std. Err = 15.562488, P-value = 0.2710
    - dummy_minute_pub_date: Coef = 0.248548, Std. Err = 0.070487, P-value = 0.0004
------------------------------------------------------------
#### Results for Maturity (Vertex): v_63
Best alpha (λ) found by cross-validation: 0.007708
Adjusted R squared of the final OLS model: 0.0028

  Final OLS Results:
    - const: Coef = -3.981838, Std. Err = 0.011851, P-value = 0.0000
    - optimism_0: Coef = -13.622287, Std. Err = 14.809615, P-value = 0.3577
    - optimism_2: Coef = -37.184991, Std. Err = 24.491009, P-value = 0.1290
    - optimism_3: Coef = -13.222695, Std. Err = 7.375856, P-value = 0.0731
    - optim

#### Regressions on uncertainty index

In [12]:
df_uncertainty = pd.read_excel("./data/processed/index_uncertainty.xlsx", index_col=0)
df_uncertainty = df_uncertainty.drop(columns=['minute'])
df_uncertainty.columns = [f"uncertainty_{i}" for i in range(len(df_uncertainty.columns))]

df_uncertainty_dif = df_uncertainty.diff()

In [13]:
df_reg_unc = mw_volatility_5.copy()
df_reg_unc = df_reg_unc.join(df_uncertainty_dif)
cols_unc = [col for col in df_reg_unc.columns if 'uncertainty_' in col]
df_reg_unc[cols_unc] = df_reg_unc[cols_unc].fillna(0)

In [14]:
results_unc = []
possible_y = vertix_names
all_x_reg_unc = cols_unc + ["dummy_minute_pub_date"]
X_reg_unc_with_dummy = df_reg_unc[all_x_reg_unc]
X_reg_unc = X_reg_unc_with_dummy[cols_unc]

scaler_reg4 = StandardScaler()
X_reg_unc_scaled = scaler_reg4.fit_transform(X_reg_unc)

X_reg_unc_scaled = pd.DataFrame(X_reg_unc_scaled, index=X_reg_unc.index, columns=X_reg_unc.columns)

for maturity in possible_y:
    Y = df_reg_unc[maturity]

    lasso_cv_model = LassoCV(cv=10, random_state=100, max_iter=10000)
    lasso_cv_model.fit(X_reg_unc_scaled, Y)

    selected_coeffs_mask = lasso_cv_model.coef_ != 0
    selected_variables_names = X_reg_unc.columns[selected_coeffs_mask].tolist()


    X_selected = X_reg_unc_with_dummy[selected_variables_names + ["dummy_minute_pub_date"]]
    X_selected_sm = sm.add_constant(X_selected)

    ols_final_model = sm.OLS(Y, X_selected_sm).fit()

    all_var_names_with_const = X_selected_sm.columns.tolist()
    ols_params = pd.Series(ols_final_model.params, index=all_var_names_with_const)
    ols_std = pd.Series(ols_final_model.bse, index=all_var_names_with_const)
    ols_pvalues = pd.Series(ols_final_model.pvalues, index=all_var_names_with_const)

    results_unc.append({
        'Vertix': maturity,
        'Selected_Alpha': lasso_cv_model.alpha_,
        'Coefs_OLS_Final': ols_params,
        'Std_OLS_Final': ols_std,
        'Pvalues_OLS_Final': ols_pvalues,
        'Adj_R_Sqrd_OLS_Final': ols_final_model.rsquared_adj
    })



In [15]:
for result in results_unc:
    print(f"#### Results for Maturity (Vertex): {result['Vertix']}")
    print(f"Best alpha (λ) found by cross-validation: {result['Selected_Alpha']:.6f}")
    print(f"Adjusted R squared of the final OLS model: {result['Adj_R_Sqrd_OLS_Final']:.4f}")
            
    # --- Final OLS Model Results ---
    if not result['Coefs_OLS_Final'].empty:
        print("\n  Final OLS Results:")
        for var in result['Coefs_OLS_Final'].index:
            coef = result['Coefs_OLS_Final'][var]
            std_err = result['Std_OLS_Final'][var]
            pval = result['Pvalues_OLS_Final'][var]
            print(f"    - {var}: Coef = {coef:.6f}, Std. Err = {std_err:.6f}, P-value = {pval:.4f}")
    else:
        print("No variables selected.")
    print("-" * 60)

#### Results for Maturity (Vertex): v_21
Best alpha (λ) found by cross-validation: 0.009377
Adjusted R squared of the final OLS model: 0.0022

  Final OLS Results:
    - const: Coef = -4.562400, Std. Err = 0.012936, P-value = 0.0000
    - uncertainty_5: Coef = 26.679933, Std. Err = 19.474697, P-value = 0.1707
    - dummy_minute_pub_date: Coef = 0.248836, Std. Err = 0.070495, P-value = 0.0004
------------------------------------------------------------
#### Results for Maturity (Vertex): v_63
Best alpha (λ) found by cross-validation: 0.011623
Adjusted R squared of the final OLS model: 0.0020

  Final OLS Results:
    - const: Coef = -3.981838, Std. Err = 0.011855, P-value = 0.0000
    - uncertainty_0: Coef = -11.378907, Std. Err = 12.326982, P-value = 0.3560
    - dummy_minute_pub_date: Coef = 0.225746, Std. Err = 0.064621, P-value = 0.0005
------------------------------------------------------------
#### Results for Maturity (Vertex): v_126
Best alpha (λ) found by cross-validation: 0.0